In [1]:
import json
from dataclasses import dataclass
from typing import Any, Dict

import pydantic
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import ConsoleSpanExporter, SimpleSpanProcessor
from opentelemetry.trace import Status, StatusCode, get_current_span

from openinference.instrumentation import (
    TracerProvider,
    get_input_value_and_mime_type,
    get_output_value_and_mime_type,
    get_span_kind,
    get_tool_attributes,
    suppress_tracing,
    using_attributes,
)
from openinference.semconv.resource import ResourceAttributes

In [2]:
endpoint = "http://127.0.0.1:6006/v1/traces"
resource = Resource(attributes={ResourceAttributes.PROJECT_NAME: "openinference-tracer"})
tracer_provider = TracerProvider(resource=resource)
tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))
tracer_provider.add_span_processor(SimpleSpanProcessor(ConsoleSpanExporter()))
tracer = tracer_provider.get_tracer(__name__)

## Chains

### Context Manager

In [ ]:
with tracer.start_as_current_span(
    "chain-span-with-plain-text-io",
    openinference_span_kind="chain",
) as span:
    span.set_input("input")
    span.set_output("output")
    span.set_status(Status(StatusCode.OK))

In [ ]:
with tracer.start_as_current_span(
    "chain-span-with-json-io",
    openinference_span_kind="chain",
) as span:
    span.set_input(
        {"input-key": "input-value"},
    )
    span.set_output(
        json.dumps({"output-key": "output-value"}),
        mime_type="application/json",
    )
    span.set_status(Status(StatusCode.OK))

In [ ]:
with tracer.start_as_current_span(
    "chain-span-with-attribute-getters",
    attributes={
        **get_span_kind("chain"),
        **get_input_value_and_mime_type("input"),
    },
) as span:
    span.set_attributes(get_output_value_and_mime_type("output"))
    span.set_status(Status(StatusCode.OK))

In [ ]:
class InputModel(pydantic.BaseModel):
    input: str


@dataclass
class OutputModel:
    output: str


with tracer.start_as_current_span(
    "chain-span-with-pydantic-input-and-dataclass-output",
    openinference_span_kind="chain",
) as span:
    span.set_input(InputModel(input="input"))
    span.set_output(OutputModel(output="output"))
    span.set_status(Status(StatusCode.OK))

### Decorator

In [ ]:
@tracer.chain
def decorated_chain_with_plain_text_output(input: str) -> str:
    return "output"


decorated_chain_with_plain_text_output("input")

In [ ]:
@tracer.chain
def decorated_chain_with_json_output(input: str) -> Dict[str, Any]:
    return {"output": "output"}


decorated_chain_with_json_output("input")

In [ ]:
@tracer.chain()
def decorated_chain_with_no_parameters(input: str) -> Dict[str, Any]:
    return {"output": "output"}


decorated_chain_with_no_parameters("input")

In [ ]:
@tracer.chain(name="decorated-chain-with-overriden-name")
def this_name_should_be_overriden(input: str) -> Dict[str, Any]:
    return {"output": "output"}


this_name_should_be_overriden("input")

In [ ]:
def chain_with_decorator_applied_as_function(input: str) -> Dict[str, Any]:
    return {"output": "output"}


decorated = tracer.chain(chain_with_decorator_applied_as_function)
decorated("input")

In [ ]:
def this_name_should_be_overriden_with_decorator_applied_as_function_with_parameters(
    input: str,
) -> Dict[str, Any]:
    return {"output": "output"}


decorated = tracer.chain(
    name="decorated-chain-with-decorator-applied-as-function-with-overriden-name"
)(this_name_should_be_overriden_with_decorator_applied_as_function_with_parameters)
decorated("input")

In [ ]:
@tracer.chain
async def decorated_async_chain(input: str) -> str:
    return "output"


await decorated_async_chain("input")  # type: ignore[top-level-await]

In [ ]:
@tracer.chain
def decorated_chain_with_error(input: str) -> str:
    raise ValueError("error")


try:
    decorated_chain_with_error("input")
except ValueError as e:
    print(e)

In [ ]:
@tracer.chain
def decorated_chain_with_child_span(input: str) -> str:
    with tracer.start_as_current_span(
        "child-span",
        openinference_span_kind="chain",
        attributes=get_input_value_and_mime_type("child-span-input"),
    ) as child_span:
        output = "output"
        child_span.set_output(output)
        child_span.set_status(Status(StatusCode.OK))
        return output


decorated_chain_with_child_span("input")

In [ ]:
@tracer.chain
def decorated_chain_with_child_span_error(input: str) -> str:
    with tracer.start_as_current_span(
        "child-span",
        openinference_span_kind="chain",
        attributes=get_input_value_and_mime_type("child-span-input"),
    ):
        raise ValueError("error")


try:
    decorated_chain_with_child_span_error("input")
except ValueError as e:
    print(e)

In [ ]:
class ChainRunner:
    @tracer.chain
    def decorated_chain_method(self, input1: str, input2: str) -> str:
        return "output"


chain_runner = ChainRunner()
chain_runner.decorated_chain_method("input1", "input2")

In [ ]:
@tracer.chain
def decorated_chain_with_input_and_output_set_inside_the_wrapped_function(input: str) -> str:
    span = get_current_span()
    span.set_input("overridden-input")  # type: ignore[attr-defined]
    span.set_output("overridden-output")  # type: ignore[attr-defined]
    return "output"


decorated_chain_with_input_and_output_set_inside_the_wrapped_function("input")

### Suppress Tracing

In [19]:
with suppress_tracing():
    with tracer.start_as_current_span(
        "THIS-SPAN-SHOULD-NOT-BE-TRACED",
        openinference_span_kind="chain",
    ) as span:
        span.set_input("input")
        span.set_output("output")
        span.set_status(Status(StatusCode.OK))

In [20]:
@tracer.chain
def decorated_chain_with_suppress_tracing(input: str) -> str:
    return "output"


with suppress_tracing():
    decorated_chain_with_suppress_tracing("input")

### Context Attributes

In [ ]:
with using_attributes(session_id="123"):
    with tracer.start_as_current_span(
        "chain-span-with-context-attributes",
        openinference_span_kind="chain",
    ) as span:
        span.set_input("input")
        span.set_output("output")
        span.set_status(Status(StatusCode.OK))

In [ ]:
@tracer.chain
def decorated_chain_with_context_attributes(input: str) -> str:
    return "output"


with using_attributes(session_id="123"):
    decorated_chain_with_context_attributes("input")

## Agents

### Context Managers

In [ ]:
with tracer.start_as_current_span(
    "agent-span-with-plain-text-io",
    openinference_span_kind="agent",
) as span:
    span.set_input("input")
    span.set_output("output")
    span.set_status(Status(StatusCode.OK))

### Decorators

In [ ]:
@tracer.agent
def decorated_agent(input: str) -> str:
    return "output"


decorated_agent("input")

## Tools

### Context Managers

In [ ]:
with tracer.start_as_current_span(
    "tool-span",
    openinference_span_kind="tool",
) as span:
    span.set_input("input")
    span.set_output("output")
    span.set_tool(
        name="tool-name",
        description="tool-description",
        parameters={"input": "input"},
    )
    span.set_status(Status(StatusCode.OK))

In [ ]:
with tracer.start_as_current_span(
    "tool-span-with-getter",
    openinference_span_kind="tool",
) as span:
    span.set_attributes(
        get_tool_attributes(
            name="tool-name",
            description="tool-description",
            parameters={"input": "input"},
        )
    )
    span.set_status(Status(StatusCode.OK))

In [ ]:
@tracer.tool
def decorated_tool(input1: str, input2: int) -> None:
    """
    tool-description
    """


decorated_tool("input1", 1)

In [ ]:
@tracer.tool
async def decorated_tool_async(input1: str, input2: int) -> None:
    """
    tool-description
    """


await decorated_tool_async("input1", 1)  # type: ignore[top-level-await]

In [ ]:
@tracer.tool(
    name="decorated-tool-with-overriden-name",
    description="overriden-tool-description",
)
def this_tool_name_should_be_overriden(input1: str, input2: int) -> None:
    """
    this tool description should be overriden
    """


this_tool_name_should_be_overriden("input1", 1)

In [ ]:
@tracer.tool
def tool_with_changes_inside_the_wrapped_function(input1: str, input2: int) -> str:
    span = get_current_span()
    print(type(span))
    span.set_input("inside-input")  # type: ignore[attr-defined]
    span.set_output("inside-output")  # type: ignore[attr-defined]
    span.set_tool(  # type: ignore[attr-defined]
        name="inside-tool-name",
        description="inside-tool-description",
        parameters={"inside-input": "inside-input"},
    )
    return "output"


tool_with_changes_inside_the_wrapped_function("input1", 1)